# TCGA-UCEC — Notebook 04
## Construction de la matrice d'expression au niveau patient

Objectif : définir une représentation unique de l'expression génique par patient,
condition nécessaire avant toute modélisation supervisée ou non supervisée.

## 1. Imports et configuration

In [1]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import warnings                                 # Masquer warnings (dépréciation)
import gc                                       # Gestion mémoire (nettoyage objets inutilisés)
import json                                     # Lecture du dictionnaire de métadonnées

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

# ==========================================================================================================
PROJECT_ROOT = r"C:\Z\M2_AIDA\TCGA_UCEC_project" #Laïla

DIRS = {
    # Racine data
    "DATA": os.path.join(PROJECT_ROOT, "data"),

    # Données
    "RAW": os.path.join(PROJECT_ROOT, "data", "raw"),

    # Données intermédiaires (persistées)
    "PROCESSED":        os.path.join(PROJECT_ROOT, "data", "processed"),
    "NORM":             os.path.join(PROJECT_ROOT, "data", "processed", "normalized"),
    "QC_FILTERED":      os.path.join(PROJECT_ROOT, "data", "processed", "qc_filtered"),
    "COHORT_FILTERED":  os.path.join(PROJECT_ROOT, "data", "processed", "cohort_filtered"),
    "SC_OBJECTS":       os.path.join(PROJECT_ROOT, "data", "processed", "single_cell_objects"),
    "PSEUDOBULK_PROC":  os.path.join(PROJECT_ROOT, "data", "processed", "pseudobulk"),

    # Artefacts (par étape du pipeline)
    "ARTEFACTS": os.path.join(PROJECT_ROOT, "data", "artefacts"),

    "EDA":         os.path.join(PROJECT_ROOT, "data", "artefacts", "exploratory_data_analysis"),
    "QC":          os.path.join(PROJECT_ROOT, "data", "artefacts", "qc_analysis"),
    "COHORT":      os.path.join(PROJECT_ROOT, "data", "artefacts", "cohort_selection"),
    "SINGLE_CELL": os.path.join(PROJECT_ROOT, "data", "artefacts", "single_cell_analysis"),
    "PSEUDOBULK":  os.path.join(PROJECT_ROOT, "data", "artefacts", "pseudobulk_preparation"),
    "DE":          os.path.join(PROJECT_ROOT, "data", "artefacts", "differential_expression"),
    "ENRICH":      os.path.join(PROJECT_ROOT, "data", "artefacts", "functional_enrichment"),

    # Autres dossiers du projet
    "RESULTS_R": os.path.join(PROJECT_ROOT, "Results_R_Analysis"),
    "TMP":       os.path.join(PROJECT_ROOT, "tmp_cache"),
    "DOCS":      os.path.join(PROJECT_ROOT, "documentation"),
}
for path in DIRS.values():
    os.makedirs(path, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ==========================================================================================================
warnings.filterwarnings("ignore") 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (10, 6) 

# ==========================================================================================================
print(f"✅ Environnement chargé. Working directory: {os.getcwd()}")


✅ Environnement chargé. Working directory: c:\Z\M2_AIDA\TCGA_UCEC_project


## 1. Chargement des données pré-traitées

Les données chargées ici correspondent aux sorties du Notebook 03
(QC gènes + normalisation).

In [2]:
expr_filename = "expr_norm_tcga_ucec.tsv"
clin_filename = "clin_tcga_ucec.tsv"

expr_path = os.path.join(DIRS["PROCESSED"], expr_filename)
clin_path = os.path.join(DIRS["PROCESSED"], clin_filename)

expr_norm = pd.read_csv(expr_path, sep="\t", index_col=0)
clin = pd.read_csv(clin_path, sep="\t", index_col=0)

print(
    f"✅ Donnees chargees\n"
    f"   - Expression normalisee : {expr_norm.shape} (genes x echantillons)\n"
    f"   - Clinique             : {clin.shape} (patients x variables)"
)


✅ Donnees chargees
   - Expression normalisee : (31876, 585) (genes x echantillons)
   - Clinique             : (597, 78) (patients x variables)


## 2. Justification du choix : 1 patient = 1 échantillon

Dans les données TCGA, plusieurs échantillons transcriptomiques peuvent être
disponibles pour un même patient (vials, portions ou aliquots distincts).

Dans ce projet, nous faisons le choix de représenter chaque patient par **un
seul vecteur d'expression**, afin de :
- garantir l'indépendance statistique des observations,
- éviter de biaiser les modèles en sur-représentant certains patients,
- simplifier l'interprétation biologique des résultats.

Lorsque plusieurs échantillons sont disponibles pour un patient, la priorité
est donnée à l'échantillon primaire (`01A`). Ce choix est standard dans les
analyses TCGA et sera discuté ultérieurement via une analyse de sensibilité
(agrégation des échantillons par patient).

## 3. Sélection d'un échantillon par patient (priorité 01A)

In [3]:
# fixe le générateur aléatoire pour garantir la reproductibilité :
# à chaque exécution, les tirages aléatoires donnent exactement les mêmes résultats.
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Construction d'un mapping patient -> echantillons
sample_to_patient = pd.Series(expr_norm.columns, index=expr_norm.columns).str.slice(0, 12)

# DataFrame facilitant la selection
sample_df = pd.DataFrame({
    "sample_id": expr_norm.columns,
    "patient_id": sample_to_patient.values
})

# Priorite a l'echantillon primaire 01A
sample_df["is_primary"] = sample_df["sample_id"].str.contains("-01A")

selected_samples = (
    sample_df
    .sort_values(by=["patient_id", "is_primary"], ascending=[True, False])
    .drop_duplicates(subset="patient_id")
)

print(f"✅ Patients uniques selectionnes : {selected_samples.shape[0]}")


✅ Patients uniques selectionnes : 557


## 4. Construction de la matrice finale patients × gènes

In [4]:
# Filtrage de la matrice d'expression
expr_patient = expr_norm[selected_samples["sample_id"].values]
expr_patient.columns = selected_samples["patient_id"].values

# Alignement avec la clinique
expr_patient = expr_patient.loc[:, expr_patient.columns.isin(clin.index)]
clin_patient = clin.loc[expr_patient.columns]

print(
    f"✅ Matrice patient-level construite\n"
    f"   - Expression : {expr_patient.shape} (genes x patients)\n"
    f"   - Clinique   : {clin_patient.shape} (patients x variables)"
)


✅ Matrice patient-level construite
   - Expression : (31876, 557) (genes x patients)
   - Clinique   : (597, 78) (patients x variables)


## 5. Sauvegarde des données patient-level

In [5]:
expr_patient.to_csv(
    os.path.join(DIRS["PROCESSED"], "expr_patient_level_tcga_ucec.tsv"),
    sep="\t"
)

clin_patient.to_csv(
    os.path.join(DIRS["PROCESSED"], "clin_patient_level_tcga_ucec.tsv"),
    sep="\t"
)

print("✅ Donnees patient-level sauvegardees")


✅ Donnees patient-level sauvegardees


## 6. Prochaines étapes

- Construction des **labels TCGA** (POLE / MSI / CN-LOW / CN-HIGH)
- Split reproductible train / validation / test
- Baseline supervisée : **MLP minimal**
- Analyse de robustesse : agrégation des échantillons par patient